## Betfair

In [15]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
import datetime as dt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
from src.betfair.api import BetsAPI
from betfairlightweight import APIClient
uname = os.environ['betfair_username']
pword = os.environ['betfair_password']
key = os.environ['betfair_key']
bf = APIClient(uname, pword, app_key=key,
               certs='/Users/jonaspaulwestermann/Dev/marchmadness/certs',
               locale='spain', lightweight=True)
bets = BetsAPI(bf)
bets.market_catalogue

Index(['marketCount', 'marketType'], dtype='object')


,marketName,runners,totalMatched
marketId,,,
1.140978762,Moneyline,"[{'selectionId': 727607, 'runnerName': 'Northe...",0.0
1.140978180,Moneyline,"[{'selectionId': 728491, 'runnerName': 'Stony ...",0.0
1.140978472,Moneyline,"[{'selectionId': 5145595, 'runnerName': 'Long ...",0.0
1.140977744,Moneyline,"[{'selectionId': 352102, 'runnerName': 'Wake F...",0.0
1.140978326,Moneyline,"[{'selectionId': 731888, 'runnerName': 'Hartfo...",0.0
1.140978908,Moneyline,"[{'selectionId': 721825, 'runnerName': 'Alabam...",0.0
1.140979054,Moneyline,"[{'selectionId': 4353352, 'runnerName': 'Alcor...",0.0
1.140978617,Moneyline,"[{'selectionId': 5145582, 'runnerName': 'Sacra...",0.0


### Market Catalogue

In [37]:
ncaa_filter = f.market_filter(competition_ids=[ncaa_competition['competition']['id']])
max_results = int(ncaa_market_types.loc['MATCH_ODDS','marketCount'])
ncaa_market_catalogue = bf.betting.list_market_catalogue(filter=ncaa_filter, max_results=max_results,
                                                         market_projection=["RUNNER_METADATA"])
ncaa_market_catalogue = pd.DataFrame(ncaa_market_catalogue).set_index('marketId')
print(ncaa_market_catalogue.shape)
ncaa_market_catalogue.sample(3)

(8, 3)


,marketName,runners,totalMatched
marketId,,,
1.140978617,Moneyline,"[{'selectionId': 5145582, 'runnerName': 'Sacra...",0.0
1.140979054,Moneyline,"[{'selectionId': 4353352, 'runnerName': 'Alcor...",0.0
1.140977744,Moneyline,"[{'selectionId': 352102, 'runnerName': 'Wake F...",0.0


### Market Book

In [31]:
ncaa_market_book = bf.betting.list_market_book(market_ids=ncaa_market_catalogue.index,
                                               currency_code='EUR',
                                               price_projection={"priceData":["EX_BEST_OFFERS"]})
ncaa_market_book = pd.DataFrame(ncaa_market_book).set_index('marketId')
print(ncaa_market_book.shape)
ncaa_market_book.sample(3)

(8, 15)


,betDelay,bspReconciled,complete,crossMatching,inplay,isMarketDataDelayed,numberOfActiveRunners,numberOfRunners,numberOfWinners,runners,runnersVoidable,status,totalAvailable,totalMatched,version
marketId,,,,,,,,,,,,,,,
1.140978617,0,False,True,False,False,True,2,2,1,"[{'selectionId': 5145582, 'handicap': 0.0, 'st...",False,OPEN,420.66,0.0,2097439060
1.140977744,0,False,True,False,False,True,2,2,1,"[{'selectionId': 352102, 'handicap': 0.0, 'sta...",False,OPEN,701.28,0.0,2097006047
1.140978762,5,False,True,False,True,True,2,2,1,"[{'selectionId': 727607, 'handicap': 0.0, 'sta...",False,OPEN,4.99,0.0,2097397099


In [32]:
ncaa_teams = pd.concat([pd.concat([pd.Series(r) for r in row], axis=1) 
                       for row in ncaa_market_catalogue.runners.values],
                       axis=1).transpose()
ncaa_teams['runnerId'] = ncaa_teams.metadata.apply(lambda d: d['runnerId'])
ncaa_teams.drop('metadata', axis=1, inplace=True)
ncaa_teams.set_index('runnerId', inplace=True)
print(ncaa_teams.shape)
ncaa_teams.sample()

(16, 4)


,handicap,runnerName,selectionId,sortPriority
runnerId,,,,
721825,0,Alabama State,721825,1


In [34]:
ncaa_market = pd.merge(ncaa_market_catalogue, ncaa_market_book,
                       left_index=True, right_index=True,
                       suffixes=['_catalogue','_book'])

In [36]:
ncaa_market.sample().squeeze().runners_book

[{'ex': {'availableToBack': [{'price': 3.6, 'size': 39.04}],
   'availableToLay': [{'price': 4.6, 'size': 30.5},
    {'price': 4.7, 'size': 30.0}],
   'tradedVolume': []},
  'handicap': 0.0,
  'selectionId': 5145582,
  'status': 'ACTIVE',
  'totalMatched': 0.0},
 {'ex': {'availableToBack': [{'price': 1.28, 'size': 109.38},
    {'price': 1.27, 'size': 110.24}],
   'availableToLay': [{'price': 1.38, 'size': 101.5}],
   'tradedVolume': []},
  'handicap': 0.0,
  'selectionId': 2909529,
  'status': 'ACTIVE',
  'totalMatched': 0.0}]